In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_temperature_pressure import*
density_param_file = os.path.join(root_path,'iltransr/pre-trained params/density_best.params')

[04:10:29] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[04:10:31] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
dropout = 0.05
batch_size = 64

In [3]:
net = ILNet(dropout=dropout)
net.IL_encoder = model.encoder
net.IL_src_embed =  model.src_embed
net.hybridize()
print(net)

ILNet(
  (IL_src_embed): HybridSequential(
    (0): Embedding(72 -> 128, float32)
    (1): Dropout(p = 0.0, axes=())
  )
  (IL_encoder): TransformerEncoder(
    (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
    (transformer_cells): HybridSequential(
      (0): TransformerEncoderCell(
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0, axes=())
          )
          (proj_query): Dense(128 -> 128, linear)
          (proj_key): Dense(128 -> 128, linear)
          (proj_value): Dense(128 -> 128, linear)
        )
        (proj): Dense(128 -> 128, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(128 -> 1024, linear)
          (activation): Activation(relu)
          (ffn_2): Dense(1024 -> 128, linear)
          (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
        )
        (layer_norm): LayerNorm(eps=1e-

In [4]:
net.load_parameters(density_param_file,ctx=ctx)
net.initialize(init=mx.init.Xavier(magnitude=magnitude), ctx=ctx)

ValueError: Parameter 'IL_textcnn._convs.11.0.weight' loaded from file '/workspace/ILTransR/iltransr/pre-trained params/density_best.params' is not present in ParameterDict, which contains parameters . Set ignore_extra=True to ignore. 

In [ ]:
density_database = pd.read_excel('smiles.xlsx',sheet_name='To_be_calculated')

In [ ]:
train_IL_smiles = density_database['SMILES'].map(canonical_smile).map(no_split).map(preprocess)
train_T =density_database['normalized_T_dens']
train_P = density_database['normalized_P_dens']
train_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(train_IL_smiles,train_T,train_P))
predict_dataloader = get_dataloader(train_dataset)

In [ ]:
predicted = predict(net, predict_dataloader,ctx)

[20:05:28] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[20:05:30] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


In [ ]:
df = pd.DataFrame(predicted)
df.to_csv('predicted_density.csv')
print(df)

                0
0     1248.643677
1     1155.114502
2     1158.526001
3     1194.765381
4     1191.607300
...           ...
5513  1107.608765
5514  1160.845947
5515   998.279053
5516  1076.010864
5517  1108.837158

[5518 rows x 1 columns]
